In [2]:
target: str = ''
start: str = '2025-01-01'
end: str = '2025-12-31'
fixed_date: str = 'false'

In [3]:
import json
from datetime import date

import petl as etl
from utils.paths import PUBLISHED, SITE

In [4]:
TARGET = SITE / 'insights/dashboard' / target / '_data'

In [5]:
TARGET.mkdir(exist_ok=True, parents=True)

In [6]:
date_range = (
    date.fromisoformat(start),
    min(date.today(), date.fromisoformat(end)) if fixed_date == 'false' else date.fromisoformat(end)
)

In [7]:
with open(TARGET / 'timestamp.json', 'w') as f:
    json.dump(dict(zip(['start', 'end'], (str(x) for x in date_range))), f)

In [8]:
data = (
    etl
    .fromjson(PUBLISHED / 'programme/projects.json')
    .convert(['Date From', 'Date To'], etl.dateparser('%Y-%m-%d'))
    .selectge('Date To', date_range[0])
    .selectle('Date From', date_range[1])
)

In [9]:
def rowgenerator(row):
    for v in row['Producing model (from Audiences Sync)']:
        yield [v]

producing_model = etl.cat(
    data
    .rowmapmany(rowgenerator, header=['Producing model'])
    .aggregate('Producing model', len),

    data
    .selecteq('Producing model (from Audiences Sync)', [])
    .aggregate(None, len)
    .addfield('Producing model', 'UNKNOWN')
)

In [10]:
projects = {
    'count': data.nrows(),
    'producing_model': dict(producing_model.records())
}

In [11]:
with open(TARGET / 'events.json', 'w') as f:
    json.dump(
        {
            'projects': projects
        },
        f,
        indent=2
    )

## Tickets

In [12]:
ticketed_events = (
    etl
    .fromcsv(PUBLISHED / 'ticketing/event-instances.csv')
    .convert('start', etl.dateparser('%Y-%m-%d %H:%M:%S'))
    .selectrangeopenleft('start', date_range[0], date_range[1])
    .selectcontains('eventType', 'MainProgrammeEvent')
)

In [13]:
tickets_sold = (
    etl
    .fromcsv(PUBLISHED / 'ticketing/tickets.csv')
    .selecteq('geography_type', 'oslaua')
    .rightjoin(ticketed_events.cut('instance_id', 'event_name'))
    .cut('event_name', 'geography_code', 'count_of_tickets')
    .convertnumbers()
    .selectnotnone('count_of_tickets')
)

In [ ]:
ticket_summary = {
    'events': {
        'names': list(ticketed_events.cut('event_name').distinct().values('event_name')),
        'count': ticketed_events.nrows(),
    },
    'sold': {
        'total': sum(tickets_sold.values('count_of_tickets')),
        'bradford': sum(tickets_sold.selecteq('geography_code', 'E08000032').values('count_of_tickets')),
    }
}

In [15]:
with open(TARGET / 'tickets.json', 'w') as f:
    json.dump(
        ticket_summary,
        f,
        indent=2
    )

## Volunteers

In [50]:
shifts = (
    etl
    .fromcsv(PUBLISHED / 'volunteers/shifts.csv')
    .convert('date', etl.dateparser('%Y-%m-%d'))
    .selecteq('type', 'BD25 Event')
    .selectrangeopenleft('date', date_range[0], date_range[1])
    .convertnumbers(strict=False)
)

In [63]:
volunteer_shifts_summary = {
    'events': {
        'count': shifts.cut('rosterfy_event_name').distinct().nrows(),
        'names': list(shifts.cut('rosterfy_event_name').distinct().values('rosterfy_event_name')),
    },
    'shifts': {
        'count': shifts.nrows(),
        'attended': sum(shifts.values('attended')),
        'hours': sum(shifts.values('hours')),
    }
}

In [64]:
with open(TARGET / 'volunteer_shifts.json', 'w') as f:
    json.dump(
        volunteer_shifts_summary,
        f,
        indent=2
    )